In [124]:
# import libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import sklearn as sk
from __future__ import division
from sklearn.cross_validation import train_test_split
import math

from tqdm import trange
from sklearn.svm import SVC

In [125]:
# read the excel sheet 
df = pd.read_excel('./BioCode for Machine Learning Updated.xlsx')

# Read in the labels
cls = df['Classification']

# Read the DNA sequences, which are strings comprised of the letters ATCG
seq = df['Aligned Sequence']

species = df['NCBI_Genus_species']

avgdist = df['avg_min_distance']
windist = df['win_sp_distance']
bdist = df['Distance_of_Branch']
avgsim = df['avg_Similarity']

In [126]:
feats = np.vstack([avgdist, windist, bdist, avgsim, cls, species]).T

print(feats.shape)

pd_feats = pd.DataFrame(feats)
pd_feats = pd_feats.dropna()

# print(pd_feats)

feats = np.array(pd_feats)
print(feats.shape)

cls = feats[:, -2:-1]
species = feats[:, -1:]
feats = feats[:, :-2]

# for i in range(len(feats)):
#     print(feats[i])

species = species.reshape(-1)
cls = cls.reshape(-1)

print(species.shape)
print(cls.shape)
print(feats.shape)

(4459, 6)
(1354, 6)
(1354,)
(1354,)
(1354, 4)


In [127]:
# Convert DNA data to numpy array, and convert NaNs to Nones
# seq = np.array(seq.fillna('None'))

# Create a binary filter to eliminate invalid DNA sequences
valid_idx = np.array([i for i in range(len(seq)) if seq[i] != 'None'])

# Apply the filter
valid_seq = seq # [valid_idx]
cls_valid = cls # [valid_idx]
cls_valid = np.array(cls_valid)
species = species # [valid_idx]

feats = feats # [valid_idx]

In [128]:
# Seperate string into individual characters
seq_arrays = [np.array([i for i in s]) for s in valid_seq]

mat_size = len(seq_arrays)

print(len(valid_seq), len(cls_valid), mat_size)

4459 1354 4459


In [129]:
valid_labels = ['Introduced', 'Invasive', 'Indigenous']
labeled_cls = [label in valid_labels for label in cls_valid]
#labeled_cls = (valid_labels[labeled_cls] == 'Indigenous').astype(int)

print(len(labeled_cls), len(species))

# Create a filter telling us which points are valid to use for supervised training
labeled_cls = np.array(labeled_cls)
cls_valid[labeled_cls]
species_valid = species[labeled_cls]

feats = feats[labeled_cls]

1354 1354


In [130]:
# apply the filter over our features and labels
# supervised_X = approx[labeled_cls]
# full_supervised_X = valid_mat[labeled_cls]
supervised_y = cls_valid[labeled_cls]
supervised_y = (supervised_y == 'Indigenous').astype(int)

In [131]:
unique = set()
species_filter = []
for i in species_valid:
    if i not in unique:
        species_filter.append(True)
        unique.add(i)
    else:
        species_filter.append(False)

In [132]:
# supervised_X = supervised_X[species_filter]
# full_supervised_X = full_supervised_X[species_filter]
supervised_y = supervised_y[species_filter]
feats = feats[species_filter]

In [133]:
print(feats.shape)
test_train_ratio = 0.5
feats_train, feats_test, y_train, y_test = train_test_split(feats, supervised_y, test_size=test_train_ratio)

(231, 4)


In [134]:
print(feats_train.shape, feats_test.shape)

(115, 4) (116, 4)


In [136]:
print(np.mean(y_test))

0.189655172414


In [119]:
for i in range(len(feats)):
    print(feats[i])

[0.190333333 0.029 0.0953 97.27333333]
[0.149986667 0.002066667 0.0654 99.81609524]
[0.149986667 0.002066667 0.0654 99.81609524]
[0.149986667 0.002066667 0.0654 99.81609524]
[0.149986667 0.002066667 0.0654 99.81609524]
[0.203375 0.007833333 0.0974 99.502]
[0.203375 0.007833333 0.0974 99.502]
[0.24075 0.0 0.1393 99.79]
[0.106125 0.003 0.0492 99.7975]
[0.191454545 0.005581818 0.0823 99.70818182]
[0.191454545 0.005581818 0.0823 99.70818182]
[0.192 0.016 0.0904 98.46]
[0.1525 0.002607143 0.0729 99.96678571]
[0.178666667 0.002 0.0885 99.85]
[0.178666667 0.014666667 0.0724 98.58]
[0.189375 0.006694444 0.0972 99.96027778]
[0.18784375 0.021642857 0.0674 99.16142857]
[0.18784375 0.021642857 0.0674 99.16142857]
[0.18784375 0.021642857 0.0674 99.16142857]
[0.18784375 0.021642857 0.0674 99.16142857]
[0.18784375 0.001 0.0948 99.925]
[0.2128 0.0304 0.0863 96.928]
[0.11325 0.019 0.0682 98.36]
[0.214333333 0.002 0.1118 99.84666667]
[0.21 0.008 0.1003 99.24]
[0.2262 0.015 0.1039 99.634]
[0.196 0.004666

In [138]:
clf = sk.linear_model.LogisticRegression() # class_weight = {0:0.15, 1:0.85})
clf.fit(feats_train, y_train)
prediction = (clf.predict(feats_test) > 0.5)*1 #Threshold
#Prediction accuracy
print('Prediciton accuracy:', np.mean((prediction == np.array(y_test))*1))
#Coefficients used by the classifier
print("Weights:", clf.coef_)

Prediciton accuracy: 0.810344827586
Weights: [[-0.55984671  0.09872703 -0.33896843 -0.01750261]]
